In [ ]:
%%sql
CREATE OR REPLACE TABLE DimDate AS
SELECT DISTINCT
    month AS DateKey,
    to_date(month, 'yyyy-MM') AS Date, -- for Market as date table
    CAST(LEFT(month, 4) AS INT) AS Year,
    CAST(RIGHT(month, 2) AS INT) AS Month,
    CONCAT('Q', CEILING(CAST(RIGHT(month, 2) AS INT) / 3.0)) AS Quarter,
    CASE 
        WHEN CAST(RIGHT(month, 2) AS INT) IN (12, 1, 2) THEN 'Summer'
        WHEN CAST(RIGHT(month, 2) AS INT) IN (3, 4, 5) THEN 'Autumn'
        WHEN CAST(RIGHT(month, 2) AS INT) IN (6, 7, 8) THEN 'Winter'
        ELSE 'Spring'
    END AS Season
FROM HousePriceTimeSeries
ORDER BY month;


StatementMeta(, e1214ae3-f3d9-41be-958f-bc73650a30b5, 13, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [ ]:
# Validate the result of running SPARK SQL for DimDate
df_count = spark.sql("SELECT COUNT(*) as record_count FROM DimDate")
record_count = df_count.collect()[0]["record_count"]
print(f"DimDate created: {record_count} records")
# Preview 5 rows
print("DimDate 前5行:")
spark.sql("SELECT * FROM DimDate LIMIT 5").show()

StatementMeta(, e1214ae3-f3d9-41be-958f-bc73650a30b5, 14, Finished, Available, Finished)

DimDate created: 36 records
DimDate 前5行:
+-------+----------+----+-----+-------+------+
|DateKey|      Date|Year|Month|Quarter|Season|
+-------+----------+----+-----+-------+------+
|2025-01|2025-01-01|2025|    1|     Q1|Summer|
|2023-01|2023-01-01|2023|    1|     Q1|Summer|
|2024-01|2024-01-01|2024|    1|     Q1|Summer|
|2023-02|2023-02-01|2023|    2|     Q1|Summer|
|2025-02|2025-02-01|2025|    2|     Q1|Summer|
+-------+----------+----+-----+-------+------+



In [4]:
%%sql
-- Create DimHouse dimension from current SCD versions
CREATE OR REPLACE TABLE DimHouse AS
SELECT
    property_id,
    bedrooms,
    bathrooms,
    stories,
    area,
    mainroad,
    guestroom,
    basement,
    hotwaterheating,
    airconditioning,
    parking,
    prefarea,
    furnishingstatus,
    valid_from AS CurrentConfigFrom,
    valid_to AS CurrentConfigTo
FROM Houses
WHERE is_current_version = true
ORDER BY property_id;

StatementMeta(, a717bed5-9fcd-4dcd-8d63-4f41c101f653, 6, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [5]:
# Validate the result of running SPARK SQL for DimHouse
df_count = spark.sql("SELECT COUNT(*) as record_count FROM DimHouse")
record_count = df_count.collect()[0]["record_count"]
print(f"DimHouse created: {record_count} records")
# Preview 5 rows
print("DimHouse 前5行:")
spark.sql("SELECT * FROM DimHouse LIMIT 5").show()

StatementMeta(, a717bed5-9fcd-4dcd-8d63-4f41c101f653, 7, Finished, Available, Finished)

DimHouse created: 542 records
DimHouse 前5行:
+-----------+--------+---------+-------+----+--------+---------+--------+---------------+---------------+-------+--------+----------------+-----------------+---------------+
|property_id|bedrooms|bathrooms|stories|area|mainroad|guestroom|basement|hotwaterheating|airconditioning|parking|prefarea|furnishingstatus|CurrentConfigFrom|CurrentConfigTo|
+-----------+--------+---------+-------+----+--------+---------+--------+---------------+---------------+-------+--------+----------------+-----------------+---------------+
|          0|       4|        2|      3|7420|       1|        0|       0|              0|              1|      2|       1|               2|          2023-01|        2025-12|
|          1|       4|        4|      4|8960|       1|        0|       0|              0|              1|      3|       0|               2|          2023-01|        2025-12|
|          2|       3|        2|      2|9960|       1|        0|       1|             

In [6]:
%%sql

-- Create FactAverageHousePrice fact table
-- Business requirement: average price for a house over the time series
CREATE OR REPLACE TABLE FactAverageHousePrice AS
SELECT 
    dh.property_id,
    dd.DateKey,
    
    -- Average price for each house at each time period
    AVG(ht.price) AS AveragePrice,
    
    -- Supporting metrics for analysis
    MIN(ht.price) AS MinPrice,
    MAX(ht.price) AS MaxPrice,
    COUNT(*) AS DataPointCount,
    AVG(ht.area) AS AverageArea,
    
    -- Current house attributes for the time period
    dh.bedrooms,
    dh.bathrooms,
    dh.area,
    
    -- Technical metadata
    CURRENT_TIMESTAMP AS ETLTimestamp
    
FROM HousePriceTimeSeries ht
INNER JOIN DimHouse dh ON ht.property_id = dh.property_id
INNER JOIN DimDate dd ON ht.month = dd.DateKey
GROUP BY dh.property_id, dd.DateKey, dh.bedrooms, dh.bathrooms, dh.area
ORDER BY dh.property_id, dd.DateKey;

StatementMeta(, a717bed5-9fcd-4dcd-8d63-4f41c101f653, 8, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
# Validate the result of running SPARK SQL for FactAverageHousePrice
df_count = spark.sql("SELECT COUNT(*) as record_count FROM FactAverageHousePrice")
record_count = df_count.collect()[0]["record_count"]
print(f"FactAverageHousePrice created: {record_count} records")
# Preview 5 rows
print("FactAverageHousePrice 前5行:")
spark.sql('SELECT * FROM FactAverageHousePrice LIMIT 5').show()

StatementMeta(, a717bed5-9fcd-4dcd-8d63-4f41c101f653, 9, Finished, Available, Finished)

FactAverageHousePrice created: 19512 records
FactAverageHousePrice 前5行:
+-----------+-------+---------------+-----------+-----------+--------------+-----------+--------+---------+----+--------------------+
|property_id|DateKey|   AveragePrice|   MinPrice|   MaxPrice|DataPointCount|AverageArea|bedrooms|bathrooms|area|        ETLTimestamp|
+-----------+-------+---------------+-----------+-----------+--------------+-----------+--------+---------+----+--------------------+
|          0|2023-01|13401205.470000|13401205.47|13401205.47|             1|     7420.0|       4|        2|7420|2026-01-09 07:48:...|
|          0|2023-02|13478007.690000|13478007.69|13478007.69|             1|     7420.0|       4|        2|7420|2026-01-09 07:48:...|
|          0|2023-03|13610972.960000|13610972.96|13610972.96|             1|     7420.0|       4|        2|7420|2026-01-09 07:48:...|
|          0|2023-04|13752062.420000|13752062.42|13752062.42|             1|     7420.0|       4|        2|7420|2026-01-09 0

In [11]:
%%sql
-- Validate star schema relationships
SELECT 'Table Created: DimDate' AS Message, COUNT(*) AS Rows FROM DimDate
UNION ALL
SELECT 'Table Created: DimHouse', COUNT(*) FROM DimHouse  
UNION ALL
SELECT 'Table Created: FactAverageHousePrice', COUNT(*) FROM FactAverageHousePrice;

-- Check for Orphaned keys
SELECT 
    'Orphaned Records Check' AS CheckType,
    COUNT(CASE WHEN d.property_id IS NULL THEN 1 END) AS BadHouseIds,
    COUNT(CASE WHEN dt.DateKey IS NULL THEN 1 END) AS BadDateKeys
FROM FactAverageHousePrice f
LEFT JOIN DimHouse d ON f.property_id = d.property_id
LEFT JOIN DimDate dt ON f.DateKey = dt.DateKey;

-- Show sample business insights
SELECT 
    'Market Average Price' AS Metric,
    ROUND(AVG(AveragePrice), 2) AS Value
FROM FactAverageHousePrice;

SELECT 
    bedrooms,
    COUNT(*) AS HouseCount,
    ROUND(AVG(AveragePrice), 2) AS AvgPrice
FROM FactAverageHousePrice
GROUP BY bedrooms
ORDER BY bedrooms;

StatementMeta(, a717bed5-9fcd-4dcd-8d63-4f41c101f653, 23, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 2 fields>

<Spark SQL result set with 1 rows and 3 fields>

<Spark SQL result set with 1 rows and 2 fields>

<Spark SQL result set with 6 rows and 3 fields>